# Semantic Segmentation Models

TASK: Create Pipeline to perform semantic segmentation on point clouds of indoor spaces.    

This is because the LAYERS CASE STUDY deals with a sport facility.

WORKFLOW TO SOLVE THE PROBLEM:
1. Loading the Pretrianed Dataset that was downloaded from the https://cvg-data.inf.ethz.ch/s3dis/ 
2. Data Preprocessing.
3. This pipeline will incorporate a pretrained segmentation Point Net to get predictions for an input set of points.
4. Then open3d library will be used to search the point cloud space.
5. Convert Individual Semantic to Meshes

# IMPORT LIBRARIES

In [4]:
import os # OS module in Python provides functions for interacting with the operating system
from pathlib import Path # makes the Path class available to our program
import re    # let you check if a particular string matches a given regular expression
import glob  #  finds all the pathnames matching a specified pattern 
import random  # series of functions for or manipulating random integers.
import numpy as np # used to perform a wide variety of mathematical operations on arrays
import pandas as pd # 

import torch   #provides two high-level features: Tensor computation (like NumPy) with strong GPU acceleration. 
                #Deep neural networks built on a tape-based autograd system
import torch.nn as nn   #gives us access to some helpful neural network things
import torch.nn.functional as F     
import torchmetrics     # a collection of machine learning metrics for distributed, 
                          # scalable PyTorch models and an easy-to-use API to create custom metrics
from torchmetrics.classification import MulticlassMatthewsCorrCoef #s a measurement to measure the quality of a binary classificatio         

import open3d as op3        #is an open-source library that supports rapid development of software that 
                            # deals with 3D data
import matplotlib.pyplot as plt #is a collection of command style functions that make matplotlib work like MATLAB
from torch.utils.data import Dataset 
import h5py #provides both a high- and low-level interface to the HDF5 library from Python

%matplotlib inline


/Users/pwavodij/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/pwavodij/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <AFB7C78A-1D52-38B1-9D33-93A75FA7D528> /Users/pwavodij/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <89972BE7-3028-34DA-B561-E66870D59767> /Users/pwavodij/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [5]:
# TEMP for supressing pytorch user warnings
import warnings
warnings.filterwarnings("ignore")

# SETTING THE FOLDER PATH

In [6]:
if Path.home().name in ['pwavodij']: # Joshua : This can be adapted for any user based on the file path to folder location
    Dropbox = Path(Path.home(), 'Dropbox', 'Case_Study_LAYERS','3D_main_2')

    # Define the main folder containing the subfolders with .txt files
main_folder = Path(Dropbox, 'Reduced_Aligned_Versions')


## Extracting the Points_cloud and targets

In [7]:
# Create empty lists to store the point cloud data
point_clouds = []
targets = []
# Iterate over the subfolders in the main folder
for subfolder in main_folder.iterdir():
    # Check if the item is a directory
    if subfolder.is_dir():
        # Iterate over the HDF5 files in the subfolder and its sub-subfolders
        for hdf5_file in subfolder.rglob('*.hdf5'):
            # Load the HDF5 dataset
            with h5py.File(hdf5_file, 'r') as hf:
                # Get the points and targets datasets
                points = hf['points'][:]
                target_labels = hf['targets'][:]

            # Append the points and targets to the respective lists
            point_clouds.append(points)
            targets.append(target_labels)

# Check if any datasets were loaded
if len(point_clouds) > 0 and len(targets) > 0:
    # Convert the lists to NumPy arrays
    point_clouds = np.concatenate(point_clouds, axis=0)
    targets = np.concatenate(targets, axis=0)

    # Example usage: Print the shape of the point clouds and targets arrays
    print("Point clouds shape:", point_clouds.shape)
    print("Targets shape:", targets.shape)
else:
    print("No datasets found in the directory.")


Point clouds shape: (805188991, 3)
Targets shape: (805188991,)


## Defining Labels and Color Map

In [8]:


CATEGORIES = {
    'ceiling'  : 0, 'floor'    : 1, 'wall'     : 2, 'beam'     : 3, 
    'column'   : 4, 'window'   : 5,'door'     : 6, 'table'    : 7, 
    'chair'    : 8, 'sofa'     : 9, 'bookcase' : 10, 'board'    : 11,
    'stairs'   : 12,'clutter'  : 13
                                    }

In [9]:
# unique color map generated via
# https://mokole.com/palette.html
COLOR_MAP = {
    0  : (47, 79, 79),    # ceiling - darkslategray
    1  : (139, 69, 19),   # floor - saddlebrown
    2  : (34, 139, 34),   # wall - forestgreen
    3  : (75, 0, 130),    # beam - indigo
    4  : (255, 0, 0),     # column - red 
    5  : (255, 255, 0),   # window - yellow
    6  : (0, 255, 0),     # door - lime
    7  : (0, 255, 255),   # table - aqua
    8  : (0, 0, 255),     # chair - blue
    9  : (255, 0, 255),   # sofa - fuchsia
    10 : (238, 232, 170), # bookcase - palegoldenrod
    11 : (100, 149, 237), # board - cornflower
    12 : (255, 105, 180), # stairs - hotpink
    13 : (0, 0, 0)        # clutter - black
}

def map_colors(x):
    try:
        return COLOR_MAP[x]
    except KeyError:
        # Handle the case where target value is not found in COLOR_MAP
        # Assign a default color or specify a custom behavior
        return (0, 0, 0)  # Default color (black)

v_map_colors = np.vectorize(map_colors)
# v_map_colors = np.vectorize(lambda x : COLOR_MAP[x])

NUM_CLASSES = len(CATEGORIES)

# MODEL_ARTIFICIAL NEURAL NETWORK

## POINT CLOUD GEOMETRY

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(
    point_clouds, targets, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val_test, y_val_test, test_size=0.5, random_state=42
)


print("Training set shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("\nValidation set shapes:")
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

print("\nTest set shapes:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# point_clouds = X_test
point_clouds = X_test
pcd = op3.geometry.PointCloud()   #creates an empty point cloud object using the Point Cloud class from the open3d.geometry
pcd.points = op3.utility.Vector3dVector(np.asarray(point_clouds))   # assigns the point_clouds array to the points attribute of the pcd object

# # Access the points attribute of the point cloud object
# points = np.asarray(pcd.points)
# points.shape

In [ ]:
## assigns color values to the points in the pcd object based on the targets array
targets = np.asarray(targets)
pcd.colors = op3.utility.Vector3dVector(np.vstack(v_map_colors(targets)).T/255)


## Plot


In [58]:
# Visualization
# op3.visualization.draw_plotly(pcd)

## Pretrained ML


In [59]:
# Point_net module is the function that implements the PointNet Neural Network Workfow
from point_net import PointNetSegHead

In [60]:
# Define the hardware that the computation will run through
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # is used to determine the device (CPU or CUDA GPU) that will be used for running the computations in PyTorch.
DEVICE    
         


'cpu'

In my case I don't have a GPU rather using the CPU.    
Already I envisaged there will be alot of computational issues this will affect the entire workflow. Already it has been observed above.!!!!    

FIRST RECOMMENDATION.                
Such Deep Neural Network Workflow Should be done on a GPU or TPU

In [94]:
# feature selection hyperparameters
# Total Number of files 19096 
# You can decide the choice of dataset for training, validation and test
NUM_TRAIN_POINTS = 4096 # train/valid points
NUM_TEST_POINTS = 15000
BATCH_SIZE = 16

# loading a pre-trained PointNetSegHead model and preparing it for evaluation.

MODEL_PATH = Path(Dropbox, 'trained_models','seg_focal','seg_model_60.pth') # get intitial model architecture

model = PointNetSegHead(num_points=NUM_TEST_POINTS, m=NUM_CLASSES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')))
model.eval();


## Detection pipeline

Defining the object detection pipeline.

Predicted classes within a 0.2m radius, then if the resulting cluster contains more than 1500 points, then the clusters bounding boxe is added to a list of proposals. 


Compute the average point score for each proposed object, by taking the total number of points assigned to the object divided by the total number of evaluated in the radius.


#LAYERS PREDICTION MODEL 

Implement the searching with open3d

In [65]:
# Reshape the points into an Nx3 array
pcd_points = np.asarray(pcd.points)  # Convert to NumPy array
pcd_points = pcd_points.reshape(-1, 3)  # Reshape to Nx3

# Place them into a point cloud object
pcd = op3.geometry.PointCloud()
pcd.points = op3.utility.Vector3dVector(pcd_points)

# Initialize KD tree object
pcd_tree = op3.geometry.KDTreeFlann(pcd)

# Perform search over radius r = 0.2
[k, idx, a] = pcd_tree.search_radius_vector_3d(pcd.points[1500], 0.2)

In [66]:
# The function first checks if the length of points is greater than npoints. 
#This condition is used to determine whether downsampling is necessary. 
# If len(points) is already less than or equal to npoints, 
# downsampling is not required, and the function will return all the available indices.

def get_downsample_choices(points, npoints):
    if len(points) > npoints:
        choice = np.random.choice(len(points), npoints, replace=False)
    else:
        choice = np.random.choice(len(points), npoints, replace=True)

    return choice

In [2]:


def get_predictions(predictions, points, cat, npoints=15000, radius=0.2, M=500):
    predictions = predictions.reshape(-1).to('cpu')  # Nx1
    pcd_points = points.permute(2, 0, 1).reshape(3, -1).to('cpu').T  # Nx3

    # Downsample points
    choice = np.random.choice(len(pcd_points), npoints, replace=False)
    pcd_points = pcd_points[choice]
    predictions = predictions[choice]

    # Obtain points for the current category
    pcd_points = pcd_points[predictions == cat]

    # Place them into a point cloud object
    pcd = op3.geometry.PointCloud()
    pcd.points = op3.utility.Vector3dVector(pcd_points)

    # Initialize KD tree object
    pcd_tree = op3.geometry.KDTreeFlann(pcd)

    # Perform M proposal searches over radius
    p_idxs = np.random.choice(len(pcd_points), M, replace=False)

    # Lists to store the results
    neighbor_indices = []
    neighbor_points = []

    for p in p_idxs:
        [k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[p], radius=radius)
        neighbor_indices.append(idx)
        neighbor_points.append(pcd.points[idx])

    return neighbor_indices, neighbor_points


In [ ]:
prediction = model()

# CREATE MESH FOR POINT CLOUD CATEGORY

In [3]:
# Create an empty list to store the meshes
meshes = []
num_categories=1170
# Iterate over each category
for cat in range(num_categories):  # Replace num_categories with the actual number of categories
    # Filter points for the current category
    category_points = pcd_points[predictions == cat]

    # Create a point cloud object for the current category
    category_pcd = op3.geometry.PointCloud()
    category_pcd.points = o3d.utility.Vector3dVector(category_points)

    # Assign color to the points of the current category
    color = color_map.get(cat, [0, 0, 1])  # Blue color for unknown categories
    category_pcd.paint_uniform_color(color)

    # Create a mesh from the point cloud
    mesh = op3.geometry.TriangleMesh.create_from_point_cloud_poisson(category_pcd, depth=8)

    # Append the mesh to the list
    meshes.append(mesh)

# Visualization: Render all the meshes
o3d.visualization.draw_geometries(meshes)


# SAVE AND EXPORT MESH

In [ ]:
import pyfbx
scene = pyfbx.FbxScene()
mesh_node = scene.create_mesh_node('mesh') # Create an FBX mesh node
mesh_node.set_mesh_data(vertices, faces, normals) # Set the mesh data
scene.save('mesh.fbx') # Save the scene as an FBX file

In [67]:
# def get_predictions(predictions, points, cat, npoints=15000, radius=0.2, M=500):
#     predictions = pred_choice.reshape(-1).to('cpu') # Nx1
#     pcd_points = norm_points.permute(2, 0, 1).reshape(3, -1).to('cpu').T # Nx3

#     # downsample points
#     choice = np.random.choice(len(pcd_points), 15000, replace=False)
#     pcd_points = pcd_points[choice]
#     predictions = predictions[choice]

#     # only obtain points for current category
#     pcd_points = pcd_points[predictions == cat]

#     # place them into a point cloud object
#     pcd = op3.geometry.PointCloud()
#     pcd.points = op3.utility.Vector3dVector(pcd_points)

#     # initialize KD tree object
#     pcd_tree = op3.geometry.KDTreeFlann(pcd)

#     # perform M proposal searches over radius 
#     p_idxs = np.random.choice(len(pcd_points), M, replace=False)
#     for p in p_idxs:
#         [k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[p], radius=radius)
